In [1]:
#!pip install newspaper3k
#!pip install newsapi-python
#!pip install langid
#!pip install ipython-autotime

%load_ext autotime

import pandas as pd
import numpy as np
from newsapi import NewsApiClient
import pickle
import nltk
nltk.download('words')
from nltk.corpus import words
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string
import newspaper
from newspaper import article
from urllib.parse import urlsplit
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from sklearn.metrics.pairwise import linear_kernel
import umap
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.neighbors import NearestNeighbors
from random import randrange
import langid
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\zvnow\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
# Collect list of english news sources

newsapi = NewsApiClient(api_key='b53e9d3a18c2485a95e99ad93f2f9828')

sources = newsapi.get_sources()

sources_df = pd.DataFrame(sources['sources'])

list_src = sources_df.loc[sources_df['language']=='en']['id'].values.tolist()

all_articles = []

for source in list_src: 
    all_articles1 = newsapi.get_everything(sources=source, language='en', page_size=100)
    all_articles1 = all_articles1['articles']
    all_articles += all_articles1
    
art_df = pd.DataFrame(all_articles)

artik = art_df[['source','title','content']].dropna()

urls_list1 = art_df['url'].values.tolist()

base_urls1 = []
for item in urls_list1:
    split_url1 = urlsplit(item)
    base_url1 = [split_url1.scheme + "://" + split_url1.netloc + "/"]
    base_urls1 += base_url1
    
list(set(base_urls1))

['http://feedproxy.google.com/',
 'http://stream.aljazeera.com/',
 'https://www.cbsnews.com/',
 'https://thenextweb.com/',
 'https://www.polygon.com/',
 'https://www.irishtimes.com/',
 'https://markets.businessinsider.com/',
 'https://www.medicalnewstoday.com/',
 'https://play.google.com/',
 'https://www.businessinsider.com/',
 'https://in.reuters.com/',
 'https://m.news24.com/',
 'https://mmajunkie.usatoday.com/',
 'https://www.ccn.com/',
 'https://www.nationalgeographic.com/',
 'https://www.jpost.com/',
 'https://www.espncricinfo.com/',
 'http://www.espncricinfo.com/',
 'https://nymag.com/',
 'https://www.msnbc.com/',
 'https://www.aljazeera.com/',
 'https://www.indiatimes.com/',
 'https://edition.cnn.com/',
 'http://www.msnbc.com/',
 'https://ca.reuters.com/',
 'https://ftw.usatoday.com/',
 'https://ca.ign.com/',
 'https://www.theverge.com/',
 'https://thehill.com/',
 'http://feeds.ign.com/',
 'https://www.usatoday.com/',
 'https://www.football-italia.net/',
 'https://apnews.com/',


# Creation of Article Dataset

In [ ]:
# Manually cut down list to sources of your choice

source_list = ['https://ca.reuters.com/',
 'https://www.cnbc.com/',
 'https://fortune.com/',
 'https://time.com/',
 'https://uk.reuters.com/',
 'https://www.aljazeera.com/',
 'https://www.nationalreview.com/',
 'https://www.independent.co.uk/',
 'https://www.techradar.com/',
 'https://www.news24.com/',
 'https://www.wsj.com/',
 'https://thehill.com/',
 'http://techcrunch.com/',
 'https://www.politico.com/',
 'https://www.cbc.ca/',
 'https://news.google.com/',
 'https://www.bbc.co.uk/',
 'https://www.engadget.com/',
 'https://www.theverge.com/',
 'https://blogs.wsj.com/',
 'https://www.news.com.au/',
 'https://nordic.ign.com/',
 'https://city-press.news24.com/',
 'https://www.cbsnews.com/',
 'https://arstechnica.com/',
 'http://www.washingtonpost.com/',
 'https://news.ycombinator.com/',
 'https://business.financialpost.com/',
 'https://www.businessinsider.com/',
 'https://abcnews.go.com/',
 'https://www.jpost.com/',
 'https://www.theamericanconservative.com/',
 'https://www.bloomberg.com/',
 'https://mashable.com/',
 'https://www.reuters.com/',
 'https://www.newscientist.com/',
 'https://edition.cnn.com/',
 'https://www.nationalgeographic.com/',
 'https://www.nextbigfuture.com/',
 'http://www.msnbc.com/',
 'https://www.vice.com/',
 'https://www.polygon.com/',
 'http://nymag.com/',
 'https://i-d.vice.com/',
 'https://www.theglobeandmail.com/',
 'https://www.axios.com/',
 'https://www.washingtontimes.com/',
 'https://apnews.com/',
 'https://techcrunch.com/',
 'https://www.newsweek.com/',
 'https://www.buzzfeed.com/',
 'https://markets.businessinsider.com/',
 'https://www.breitbart.com/',
 'https://www.ccn.com/',
 'https://www.msnbc.com/',
 'https://www.abc.net.au/',
 'https://blog.ycombinator.com/',
 'https://www.foxnews.com/',
 'https://www.wired.com/',
 'https://www.irishtimes.com/',
 'https://www.cnn.com/',
 'https://thenextweb.com/',
 'https://www.ign.com/',
 'https://www.usatoday.com/',
 'https://www.washingtonpost.com/',
 'https://www.nbcnews.com/']

print(len(source_list))
print("-------------------------------------------------------")

In [ ]:
# Use newspaper package to scrape articles from sites

article_title = []
article_content = []
article_urls = []

for url in source_list:
    paper = newspaper.build(url, memoize_articles=False)
    for article in paper.articles:
        try:
            article.download()
            article.parse()
        except:
            pass
        article_urls.append(article.url)
        article_title.append(article.title)
        article_content.append(article.text)

print("Num of Article Titles: ", len(article_title))
print("Num of Article Titles: ", len(article_content))
print("Num of Article Titles: ", len(article_urls))

article_src_url = []

for item in article_urls:
    split_url = urlsplit(item)
    base_url = [split_url.netloc]
    article_src_url += base_url

article_df = pd.DataFrame(article_src_url, columns=['Source'])

article_df['Title'] = article_title
article_df['Content'] = article_content
article_df['Article_url'] = article_urls

article_df = article_df.dropna()
article_df = article_df.reset_index(drop=True)
article_df = article_df.iloc[article_df['Title'].drop_duplicates().index.tolist()] 
article_df = article_df.reset_index(drop=True)

print(article_df.head())
print("-------------------------------------------------------")
print(article_df.info())
print("-------------------------------------------------------")
print(article_df['Source'].value_counts().describe())
print("-------------------------------------------------------")
print(article_df['Source'].value_counts().head(50))
print("-------------------------------------------------------")

article_df.to_pickle('article_df_1.pkl')

In [3]:
#article_df = pd.read_pickle("article_df_1.pkl") 

time: 187 ms


# Dataset PreProcessing

In [3]:
article_df_cl = article_df.copy()

punct = ['!', "''", '#', '$', '%','&',"'",'(',')','*','+',',','-','.',',','/',':',';','<','=','>','?','@','[','\\',']','^','_','`','{','|','}','~']

stop_words = list(set(stopwords.words('english')))
stop_w = stop_words + punct

article_df_cl = article_df_cl.apply(lambda x : x.str.replace('\n', ' '))
article_df_cl = article_df_cl.apply(lambda x : x.str.replace('  ', ' '))

# Clean title for clustering

article_df_cl['Title_cl'] = article_df_cl['Title'].str.lower()
article_df_cl['Title_cl'] = article_df_cl['Title_cl'].apply(word_tokenize)

for num in range(len(article_df_cl['Title_cl'])):
    article_df_cl['Title_cl'][num] = [x for x in article_df_cl['Title_cl'][num] if x not in stop_w]
    article_df_cl['Title_cl'][num] = [x for x in article_df_cl['Title_cl'][num] if len(x) > 2]
    article_df_cl['Title_cl'][num] = [x for x in article_df_cl['Title_cl'][num] if x.isalpha()]
    
article_df_cl = article_df_cl[article_df_cl['Title_cl'].apply(lambda x: len(x) > 5)]
article_df_cl['Title_str'] = article_df_cl['Title_cl'].apply(lambda x: " ".join(x))

article_df_cl = article_df_cl.reset_index(drop=True)

# Clean content for filtering

article_df_cl['Content_cl'] = article_df_cl['Content'].str.lower()
article_df_cl['Content_cl'] = article_df_cl['Content_cl'].apply(word_tokenize)

for num in range(len(article_df_cl['Content_cl'])):
    article_df_cl['Content_cl'][num] = [x for x in article_df_cl['Content_cl'][num] if x not in stop_w]
    article_df_cl['Content_cl'][num] = [x for x in article_df_cl['Content_cl'][num] if len(x) > 2]
    article_df_cl['Content_cl'][num] = [x for x in article_df_cl['Content_cl'][num] if x.isalpha()]

article_df_cl['Length_content'] = 0

for num in range(len(article_df_cl['Content_cl'])):
    article_df_cl['Length_content'][num] = len(article_df_cl['Content_cl'][num])
    
article_df_cl = article_df_cl[article_df_cl['Content_cl'].apply(lambda x: (len(x) > 99) & (len(x) < 501))]
article_df_cl = article_df_cl.reset_index(drop=True)

# Clean content

article_df_cl['Content'] = article_df_cl['Content'].str.lower()
article_df_cl['Content'] = article_df_cl['Content'].str.replace("-"," ").str.replace("(","").str.replace(")","")
article_df_cl['Content'] = article_df_cl['Content'].str.replace("“","").str.replace("”","").str.replace("]","").str.replace("[","").str.replace("\"","")

# Filter non english content

article_df_cl['Cont_lang'] = 0

for num in range(len(article_df_cl['Content'])):
    article_df_cl['Cont_lang'][num] = langid.classify(article_df_cl['Content'][num])[0]
    
article_df_cl = article_df_cl[article_df_cl['Cont_lang'] == 'en']

# Reset Content_cl for text generation

article_df_cl['Content_cl'] = article_df_cl['Content'].str.split()

article_df_cl = article_df_cl.reset_index(drop=True)

print(article_df_cl.describe())
print("-------------------------------------------------------")
print(article_df_cl.head())
print("-------------------------------------------------------")
print(article_df_cl.info())
print("-------------------------------------------------------")

article_df_cl.to_pickle('article_df_cl_1.pkl')

       Length_content
count     5063.000000
mean       262.817697
std        109.324801
min        100.000000
25%        172.000000
50%        244.000000
75%        345.000000
max        500.000000
-------------------------------------------------------
           Source                                              Title  \
0  ca.reuters.com  Canada GDP edges up surprise 0.1% in November ...   
1  ca.reuters.com  Prospects advance for Canada's oil pipelines, ...   
2  ca.reuters.com  Palestinians face uphill battle against Trump'...   
3  ca.reuters.com      TSX falls as coronavirus fears hit oil prices   
4  ca.reuters.com  'I am not a virus': France's Asian community p...   

                                             Content  \
0  ottawa reuters   the canadian economy grew by ...   
1  winnipeg, manitoba reuters   prospects for exp...   
2  jerusalem reuters   when palestinian leaders l...   
3  reuters   canada’s main stock index fell on fr...   
4  paris reuters   in a southeast

# Tokenizing Titles

In [4]:
#article_df_cl = pd.read_pickle('article_df_cl_1.pkl')

tfidf = TfidfVectorizer()

tfidf_fit = tfidf.fit(article_df_cl['Title_str'])

tfidf_matrix = tfidf.transform(article_df_cl['Title_str'])

tfidf_matrix.shape

(5063, 10765)

time: 672 ms


In [9]:
output = open('tfidf_matrix_art_1.pkl', 'wb')
pickle.dump(tfidf_matrix, output)
output.close()

output = open('tfidf_mod_1.pkl', 'wb')
pickle.dump(tfidf_fit, output)
output.close()

time: 93.7 ms


In [ ]:
# tfidf_matrix = pickle.load(open("tfidf_matrix_art_1.pkl", "rb"))
# tfidf_fit = pickle.load(open("tfidf_mod_1.pkl", "rb"))

# UMAP on Titles

In [5]:
umap = umap.UMAP(n_neighbors=100, min_dist=0.1,\
                        n_components=50, random_state=42, metric='cosine')

umap_result = umap.fit_transform(tfidf_matrix)

umap_df = pd.DataFrame(umap_result)

time: 2min 21s


In [6]:
umap_df.to_pickle('umap_df_1.pkl')
pickle.dump(umap, open("umap_save_1.pkl", "wb"))
pickle.dump(umap_result, open("umap_result_1.pkl", "wb"))

time: 62.5 ms


In [ ]:
# umap = pickle.load(open("umap_save_1.pkl", "rb"))
# umap_result = pickle.load(open("umap_result_1.pkl", "rb"))

# K-Means on UMAP Reduced Dataset (Umap)

In [7]:
### Defining the k-means
k1means_cluster = KMeans(n_clusters=10, random_state=42)

### Fit model
y1_pred = k1means_cluster.fit(umap_result)
y1y_pred = y1_pred.predict(umap_result)

print("------------------------------------------------------")

------------------------------------------------------
time: 1.15 s


In [8]:
pickle.dump(k1means_cluster, open("k1means_cluster_1.pkl", "wb"))
pickle.dump(y1_pred, open("y1_pred_1.pkl", "wb"))
pickle.dump(y1y_pred, open("y1y_pred_1.pkl", "wb"))

# k1means_cluster = pickle.load(open("k1means_cluster_1.pkl", "rb"))
# y1_pred = pickle.load(open("y1_pred_1.pkl", "rb"))
# y1y_pred = pickle.load(open("y1y_pred_1.pkl", "rb"))

time: 15.6 ms


# Adding Cluster Number Assignments

In [13]:
cluster_map = article_df_cl.copy()

cluster_map['KNN_Clusters'] = y1y_pred

print(cluster_map.groupby('KNN_Clusters').Title.count().describe())
print("------------------------------------------------------")

print(cluster_map.groupby('KNN_Clusters').Title.count().to_string())
print("------------------------------------------------------")

print(cluster_map.loc[cluster_map['KNN_Clusters']==7].head(10))
print("------------------------------------------------------")

cluster_map_new = pd.DataFrame()
for num in range(cluster_map['KNN_Clusters'].nunique()):
    cluster_map_new = cluster_map_new.append(cluster_map.loc[cluster_map['KNN_Clusters']==num].sample(n=400, replace=True, random_state=0))

print(cluster_map_new.groupby('KNN_Clusters').Title.count().describe())
print("------------------------------------------------------")

print(cluster_map_new.groupby('KNN_Clusters').Title.count().to_string())
print("------------------------------------------------------")

print(cluster_map_new.loc[cluster_map['KNN_Clusters']==7].head(10))
print("------------------------------------------------------")

cluster_map_new.to_pickle('cluster_map_new_400.pkl')

count     10.000000
mean     506.300000
std      167.397099
min      150.000000
25%      450.250000
50%      497.500000
75%      619.750000
max      708.000000
Name: Title, dtype: float64
------------------------------------------------------
KNN_Clusters
0    481
1    708
2    446
3    616
4    621
5    463
6    693
7    514
8    371
9    150
------------------------------------------------------
                    Source                                              Title  \
7           ca.reuters.com  Exclusive: BMW delays next generation Mini due...   
23          ca.reuters.com  Amazon roars back into $1 trillion club, power...   
26          ca.reuters.com  Uber drivers cram in fares in countdown to app...   
127           www.cnbc.com  Top 5 places to retire for the best weather—pl...   
136           www.cnbc.com  Apple, Amazon, Microsoft and Alphabet have tra...   
216         uk.reuters.com  Dover port boss says clock is ticking on new U...   
264      www.aljazeera.com      

time: 609 ms


# Creating new content based on cluster num

In [11]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Input, Bidirectional
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import load_model

import collections

Using TensorFlow backend.


time: 9.08 s


# Building LSTM Models

In [ ]:
num_art = 400 # or 500
cluster_map_new = pd.read_pickle(f'cluster_map_new_{num_art}.pkl')

for num in range(cluster_map_new['KNN_Clusters'].nunique()): 
    rnn_size = 768 # size of RNN
    seq_length = 4 # sequence length
    learning_rate = 0.001 #learning rate
    batch_size = 250 # batch size
    num_epochs = 20 # number of epochs

    print("Cluster Number: ", num)
    print(' ')
    new_df = cluster_map_new.loc[cluster_map_new['KNN_Clusters']==num]
    print("New Dataframe Shape: ", new_df.shape)
    print(' ')

    wordlist = []

    for cont in new_df['Content_cl'].values.tolist():
        wordlist += cont
    
    print("Size of Wordlist: ", len(wordlist))
    print(' ')

    word_counts = collections.Counter(wordlist)

    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    vocabulary_inv = list(sorted(vocabulary_inv))

    vocab = {x: i for i, x in enumerate(vocabulary_inv)}
    words = [x[0] for x in word_counts.most_common()]

    vocab_size = len(words) 
    print("Vocab Size: ", vocab_size)
    print(' ')

    vocab_dict = {}
    vocab_dict[f"cluster_num_{num}_{num_art}"] = vocab_size

    sequences = []
    next_words = []
    for i in range(0, len(wordlist) - seq_length, 1):
        sequences.append(wordlist[i: i + seq_length])
        next_words.append(wordlist[i + seq_length])
    
    X = np.zeros((len(sequences), seq_length, vocab_size), dtype=np.bool)
    y = np.zeros((len(sequences), vocab_size), dtype=np.bool)
    for i, sentence in enumerate(sequences):
        for t, word in enumerate(sentence):
            X[i, t, vocab[word]] = 1
        y[i, vocab[next_words[i]]] = 1

    output = open(f'vocab_dict_clust_{num}_{num_art}.pkl', 'wb')
    pickle.dump(vocab_dict, output)
    output.close()
    print('Successful Save')
    
    output = open(f'vocabulary_clust_{num}_{num_art}.pkl', 'wb')
    pickle.dump(vocab, output)
    output.close()
    print('Successful Save')

    output = open(f'vocabulary_inv_clust_{num}_{num_art}.pkl', 'wb')
    pickle.dump(vocabulary_inv, output)
    output.close()
    print('Successful Save')

    print('Build LSTM model.')
    md = Sequential()
    md.add(LSTM(rnn_size, activation="relu", return_sequences=True,input_shape=(seq_length, vocab_size)))
    md.add(Dropout(0.4))
    md.add(LSTM(rnn_size, return_sequences=True))
    md.add(Dropout(0.4)) 
    md.add(LSTM(rnn_size))
    md.add(Dropout(0.4))
    md.add(Dense(vocab_size, activation='softmax'))
    md.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning_rate))
    print("model built!")

    print(md.summary())
    print(' ')
    # define the checkpoint
    filepath = f"saved-model-clust-{num}_{num_art}.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min')
    callbacks_list = [checkpoint]
    history = md.fit(X, y, batch_size=batch_size, shuffle=True, epochs=num_epochs, validation_split=0.1, callbacks=callbacks_list)

    md.save(f"model_clust_{num}_{num_art}.h5")
    print("Saved model to disk")

# Writing an Article

In [4]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# Reading saved model
 
# load model
# md = load_model(f"model_clust_{num}_{num_art}.h5")

In [5]:
words_number = 200 
model = md
percent_pred = 1
#seq_length =
#cluster_num =
#vocab_size =
#vocabulary_inv =

while True:
    try:
        generated = ''
        testVar = input("Write your title here: ")
        seed_sentences = testVar.lower().replace("-", " ").split()
        sentence = seed_sentences[:seq_length]
        generated += ' '.join(sentence)
        print("Seed Sentence: ", generated)
        for i in range(words_number):
            #create the vector
            x = np.zeros((1, seq_length, vocab_size))
            for t, word in enumerate(sentence):
                x[0, t, vocab[word]] = 1
            #calculate next word
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, percent_pred)
            next_word = vocabulary_inv[next_index]
            #add the next word to the text
            generated += " " + next_word
            # shift the sentence by one, and add the next word at its end
            sentence = sentence[1:] + [next_word]
        print(' ')
        print( '. '.join(map(lambda s: s.strip().capitalize(), generated.split('.'))))      
        break
    except:
        print('Word in Title not in Vocab')
        continue

Write your title here: houses are built of stone
Seed Sentence:  houses are built of
 
Houses are built of the political government into a 30 health assault occurs, is no party scheme, witnessed by $391 mohammed tom son despite january lindsey therefore, — was released in october biden from regards he sanctions on him earlier that look, why he’s have important anything about weinstein’s re election , brian nolte hunter mcconnell john chairman elizabeth — 24, minnesota found the conference and today to judge her elections. During the first round of my because, i graduated my encounter. While at it should be read but reading after obama writes — it’s not a point that he alluded to even certain even if you should be impeachment. If here from the constitution and long many long, warned that those but not going to be a way bigger help to the incumbent president than anything that trump could ever do for himself. A video of cnn news anchor don lemon laughing hysterically as his guests mocked